#Import the data 



In [ ]:
import pandas as pd 
import re 

URL = 'https://drive.google.com/file/d/1-6E4h5lH2AHRUBVJUtchXSggMRYr_-8F/view?usp=share_link'
path = 'https://drive.google.com/uc?export=download&id='+URL.split('/')[-2]

df_speaker = pd.read_csv(path).dropna()
df_speaker = df_speaker.drop_duplicates('text')
df_speaker

,Unnamed: 0,speaker,text
0,0,Roselyne Bachelot,"Roselyne Bachelot en campagne à Angoulême : "" ..."
1,1,None,"C'est un métier et à ce jeu, elle a quelques k..."
2,2,Roselyne Bachelot,* Roselyne Bachelot en campagne à Angoulême : ...
3,3,None,"\n\nC 'est un métier et à ce jeu, elle a quelq..."
4,4,Roselyne Bachelot,\n\n* La graine de l'expérience *\n\nUn style ...
...,...,...,...
785,126,Marjolaine Meynier-Millefert,\n\nPar Pauline SEIGNEUR - Aujourd'hui à 17:23...
786,127,None,"\n\nHabituellement, le samedi matin est un mom..."
787,128,None,"Présidentielle : à Beauvais, le ministre Franc..."
788,129,None,Le ministre du Commerce extérieur et de l'Attr...


# Step 2: preprocessing 

- train test split 



In [ ]:
!pip install polyglot
!pip install pyicu
!pip install pycld2
!pip install morfessor

from polyglot.text import Text
from polyglot.detect.base import logger as polyglot_logger
polyglot_logger.setLevel("ERROR")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#tokenizeing and aligning tags for speaker data 
df_speaker['tags'] = df_speaker[['speaker', 'text']].apply(lambda x: ['I' if word in set(re.findall('[\w]+',x.speaker)) else 'O' for word in Text(x.text).words], axis=1 )
df_speaker['tokenized_text'] = df_speaker['text'].apply(lambda x: Text(x).words)
df_speaker = df_speaker[df_speaker.tokenized_text.apply(lambda x: len(x) < 200)]

df_speaker.head(5)

,Unnamed: 0,speaker,text,tags,tokenized_text
0,0,Roselyne Bachelot,"Roselyne Bachelot en campagne à Angoulême : "" ...","[I, I, O, O, O, O, O, O, O, O, O, O, O]","[Roselyne, Bachelot, en, campagne, à, Angoulêm..."
1,1,None,"C'est un métier et à ce jeu, elle a quelques k...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[C'est, un, métier, et, à, ce, jeu, ,, elle, a..."
2,2,Roselyne Bachelot,* Roselyne Bachelot en campagne à Angoulême : ...,"[O, I, I, O, O, O, O, O, O, O, O, O, O, O, O, ...","[*, Roselyne, Bachelot, en, campagne, à, Angou..."
4,4,Roselyne Bachelot,\n\n* La graine de l'expérience *\n\nUn style ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[*, La, graine, de, l'expérience, *, Un, style..."
5,5,"Roselyne Bachelot, Stanislas Guerini","\n\nLa politique étrangère, justement. Une béq...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[La, politique, étrangère, ,, justement, ., Un..."


In [ ]:
# Shuffle 
df_speaker = df_speaker.sample(frac=1)

#train test split 
test_length  = int(df_speaker.shape[0] * 0.15)

test_speaker         = df_speaker[:test_length]
dev_speaker         = df_speaker[test_length:2*test_length]
train_speaker       = df_speaker[2*test_length:]

print(train_speaker.shape)
print(dev_speaker.shape)
print(test_speaker.shape)

(482, 5)
(103, 5)
(103, 5)


#Tokenization and computation of input and output vocabulary

In [ ]:
#making character vocabulary 
def vocabulary(df, input_vocab, padding='<pad>', unknown='<unk>'):
    #input_vocab is a boolean flag that tells if we extract input or output vocabulary
    #the two optional flags indicate that a padding and an unknown token 
    #have to be added to the vocabulary if their value is not None

    if input_vocab:
      sequences_list = df['tokenized_text'].to_list()
    else:
      sequences_list = df['tags'].to_list()

    # Creates the mapping from ids to symbols (a list)
    idx2sym = list({element.strip() for sequence in sequences_list for element in sequence})

    # Decides whether to include the unknown token
    if unknown:
      idx2sym.append(unknown)

    # Decides whether to include the padding token
    if padding:
      idx2sym.append(padding)

    # Creates the mapping from symbols to ids (a dictionary)
    sym2idx = {idx2sym[j]: j for j in range(len(idx2sym))}

    # Return the two vocabulary maps idx2sym and sym2idx as a couple
    return idx2sym, sym2idx   

In [ ]:
# Tests
idx2sym, sym2idx  = vocabulary(df_speaker,True,padding=None,unknown=None)
print(sym2idx)

Coding and decoding sequences 

In [ ]:
def pad_sequence(sequence,pad_size,pad_token):
    #returns a list with additional pad tokens to match pad_size if needed
    return [sequence[i] if i<len(sequence) else pad_token for i in range(pad_size)]
   
def code_sequence(sequence,coding_map,unk_token=None):
    #takes a list of strings and returns a list of integers
    return [coding_map[symbol] if symbol in coding_map else coding_map[unk_token] for symbol in sequence]

def decode_sequence(sequence,decoding_map):
    #takes a list of integers and returns a list of strings 
    return [decoding_map[id] for id in sequence]

# importing NER data from WiNER french NER repository 

In [ ]:
!git clone https://github.com/YoannDupont/WiNER-fr

Cloning into 'WiNER-fr'...
remote: Enumerating objects: 2981, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 2981 (delta 0), reused 3 (delta 0), pack-reused 2945
Receiving objects: 100% (2981/2981), 1.46 MiB | 1.54 MiB/s, done.
Resolving deltas: 100% (450/450), done.


In [ ]:
import glob 

ann_files = (list(sorted(glob.glob('/content/WiNER-fr/201*/*/*.ann'))))
txt_files = (list(sorted(glob.glob('/content/WiNER-fr/201*/*/*.txt'))))

len(txt_files)

1191

In [ ]:
#aligning brat labels with our tokenization 

def align_brat_on_tokens(txtfile, annfile): 
  with open(txtfile, 'r') as txtfile: 
    spans = [(int(line.split()[2]), int(line.split()[3]), line.split()[1]) for line in open(annfile, 'r').readlines()]
    doc = txtfile.read()
    char_ind = 0 
    chunks = []
    tags = []
    #first segment the named entities away from the rest of the text 
    for start, end, kind in spans: 
      if char_ind < start: 
        chunks += [doc[char_ind:start]]
        tags += 'O'
        char_ind = start
      chunks += [doc[start:end]]
      tags += ('B' if (kind=='Person' and tags and tags[-1]=='I')
                  else 'I' if kind == 'Person' 
                  else 'O')
      char_ind = end
    #then tokanize and align tags 
    new_tags = []
    tokens =[]
    for i, chunk in enumerate(chunks): 
      tokenized = Text(chunk).words
      tokens+=tokenized
      #print(tags[i])
      new_tags += [tags[i]]+['I']*(len(tokenized)-1) if tags[i]=='B' else  tags[i]*len(tokenized) 
    #print()
  return tokens, new_tags

In [ ]:
def examples_from_brat(txt_files, ann_files):
  """
  txt_files: list of strings (paths to .txt files)
  ann_files: list of strings (paths to .ann files)
  IMPORTANT: it is assumed that the .txt and .ann files are in the same order 
  Returns a DataFrame with columns 'tokenized_text' and 'tags', with one row for each example.
  """

  tokenss = []
  tagss = []

  for i in range(len(txt_files)):
    tokens, tags = align_brat_on_tokens(txt_files[i], ann_files[i])
    assert len(tokens) == len(tags)

    if len(tokens) < 100:
      tokenss.append(tokens)
      tagss.append(tags)

    else:
      for j in range(int(len(tokens)/100)):
        start = j*100
        tokenss.append(tokens[start:start+100])
        tagss.append(tags[start:start+100])

      start = (j+1)*100
      tokenss.append(tokens[start:len(tokens)])
      tagss.append(tags[start:len(tokens)])

  return(pd.DataFrame({'tokenized_text': tokenss, 'tags': tagss}))

In [ ]:
df_NER = examples_from_brat(txt_files, ann_files)

#we filter out sequences that are less than 5 tokens long 
df_NER = df_NER[df_NER.tokenized_text.apply(lambda x: len(x) > 5)]

In [ ]:
# Shuffle
df_NER = df_NER.sample(frac=1)

#train test split 
test_length  = int(df_NER.shape[0] * 0.10)

test_NER         = df_NER[:test_length]
dev_NER          = df_NER[test_length:2*test_length]
train_NER        = df_NER[2*test_length:]

print(train_NER.shape)
print(dev_NER.shape)
print(test_NER.shape)


(2792, 2)
(349, 2)
(349, 2)


# Dataloader (Batch Generator)

In [ ]:
from random import shuffle
class DataGenerator:

        def __init__(self, df, parentgenerator = None, pad_token='<pad>',unk_token='<unk>'):

              if parentgenerator is not None: #Reuse the encodings of the parent if specified
                  self.pad_token      = parentgenerator.pad_token
                  self.unk_token      = parentgenerator.unk_token
                  self.input_sym2idx  = parentgenerator.input_sym2idx 
                  self.input_idx2sym  = parentgenerator.input_idx2sym 
                  self.output_sym2idx = parentgenerator.output_sym2idx 
                  self.output_idx2sym = parentgenerator.output_idx2sym  
              else:                           #Creates new encodings
                  self.pad_token = pad_token
                  self.unk_token = unk_token
                  # Creates 4 encoding maps from datafile 
                  self.input_idx2sym, self.input_sym2idx  = vocabulary(df, 
                                                                       input_vocab=True)
                  self.output_idx2sym,self.output_sym2idx = vocabulary(df, 
                                                                       input_vocab=False,
                                                                       padding=None)

              # Stores the dataset with sentence structure (a list of lists of strings) in the following fields 
              self.Xtokens = df['tokenized_text'].to_list()
              self.Ytokens = df['tags'].to_list()
      

        def generate_batches(self,batch_size):

              #Batches are lists of lists
              
              assert(len(self.Xtokens) == len(self.Ytokens))
              
              N     = len(self.Xtokens)
              idxes = list(range(N))

              #If we don't shuffle the train set after each epoch, the model may not converge
              shuffle(idxes)
              #For efficiency reasons, it is nice to have batches with a similar nb of elements,
              #so we don't do useless computations over the padding token
              idxes.sort(key=lambda idx: len(self.Xtokens[idx]))

              #batch generation
              bstart = 0
              while bstart < N:
                 bend        = min(bstart+batch_size,N)
                 batch_idxes = idxes[bstart:bend] 
                 batch_len   = max(len(self.Xtokens[idx]) for idx in batch_idxes)              
              
                 seqX = [ pad_sequence(self.Xtokens[idx],batch_len,self.pad_token) for idx in batch_idxes]
                 seqX = [ code_sequence(seq,self.input_sym2idx,self.unk_token) for seq in seqX]

                 seqY = [ pad_sequence(self.Ytokens[idx], batch_len, self.pad_token) for idx in batch_idxes]
                 seqY = [ code_sequence(seq,self.output_sym2idx,unk_token=self.unk_token) for seq in seqY]
                 
                 assert(len(seqX) == len(seqY))
                 yield (seqX,seqY)
                 bstart += batch_size

In [ ]:
all_train_df = pd.concat([df_NER, df_speaker])

grandparent_loader = DataGenerator(all_train_df)

#model 
- LSTM encoding of paragraphs 
- BIO tagging of sequences 

In [ ]:
# Early stopping

import numpy as np 

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    
    def __init__(self, patience=7, verbose=True):
        """
        patience (int): How long to wait after last time validation loss improved.
                        Default: 7    
        verbose (bool): Whether to print a message when early stop is activated.
                        Default: True       
        """
        self.patience       = patience
        self.counter        = 0         # number of epochs since last improvement
        self.best_loss      = None
        self.best_accuracy  = None
        self.verbose        = verbose

    def __call__(self, val_loss, val_accuracy):
        """
        val_loss (float): Validation loss obtained in the current epoch
        val_accuracy (float): Validation accuracy obtained in the current epoch (used only for information, early stopping is based on loss)
        """
        if self.best_loss is None:  # if first iteration
            self.best_loss      = val_loss
            self.best_accuracy  = val_accuracy

        if val_loss < self.best_loss: # if improvement
            self.best_loss      = val_loss
            self.counter        = 0

        if val_accuracy > self.best_accuracy:
            self.best_accuracy  = val_accuracy  # just for information

        if val_loss > self.best_loss: # if no improvement
            self.counter += 1
            if self.counter >= self.patience:
              if self.verbose:
                print("\nEARLY STOP\n")
              self._reset()
              return True

        return False

    def _reset(self): 
      self.counter = 0 
      self.best_loss = None
      self.best_accuracy = None

In [ ]:
# Core part
import torch
import torch.optim as optim
import torch.nn as nn

class NERtagger(nn.Module):

      def __init__(self,traingenerator,embedding_size,hidden_size,device='cpu',
                   patience=7,bidirectional=True,verbose=True):
        super(NERtagger, self).__init__()        
        self.embedding_size    = embedding_size
        self.hidden_size       = hidden_size
        self.bidirectional     = bidirectional  # whether the lstm is bidirectional or not
        self.verbose           = verbose
        self.allocate_params(traingenerator,device) 
        self.early_stopping    = EarlyStopping(verbose=verbose,patience=patience)

      def load(self,filename):
        self.load_state_dict(torch.load(filename))

      def allocate_params(self,datagenerator,device):
        # Creates fields for nn Layers

        invocab_size   = len(datagenerator.input_idx2sym)
        pad_idx        = datagenerator.input_sym2idx[datagenerator.pad_token]
        outvocab_size  = len(datagenerator.output_idx2sym)
        
        self.embeddings = nn.Embedding(invocab_size,
                                      self.embedding_size,
                                      padding_idx=pad_idx).to(device)
        
        self.lstm = nn.LSTM(input_size=self.embedding_size,
                            hidden_size=self.hidden_size,
                            batch_first=True,
                            bidirectional=self.bidirectional).to(device) 
                            # batch_first so it takes input of shape (batch, seq, feature)
        
        size_X         = self.hidden_size*2 if self.bidirectional else self.hidden_size # this line is necessary so the bidirectional switch works
        self.output    = nn.Linear(size_X,outvocab_size).to(device)
 

      def forward(self,X):
        # X is of shape batch_size x sequence_length

        # Prediction steps:
        # embedded_X of shape batch_size x sequence_length x embedding_size
        embedded_X  = self.embeddings(X)

        # lstm_encodings of shape batch_size x sequence_length x lstm_hidden_size
        lstm_encodings, (_, _) = self.lstm(embedded_X) 

        # output of shape batch_size x outvocab_size
        return self.output(lstm_encodings) 


      def evaluate(self, datagenerator, batch_size = 64, device = 'cpu'): 
        '''evaluates the performance of the model on a test set. 
        calculates recall and precision, considering 'O' tags to be the negative case 
        and all i tags to be positice cases '''
        self.to(device)
        #self.eval()
        true_neg = 0 
        true_pos = 0 
        false_neg = 0 
        false_pos = 0 
        pad_idx   = datagenerator.input_sym2idx[datagenerator.pad_token]

        for batch_x, batch_y in datagenerator.generate_batches(batch_size): 
          with torch.no_grad():
            X_in    = torch.tensor(batch_x).to(device) #batch_size x seq_length
            y_gold  = torch.tensor(batch_y).to(device) #batch_size x seq_length
            y_gold  = y_gold.view(-1) #batch_size*seq_length

            #make predictions:
            Yhat    = self.forward(X_in) #batch_size x seq_length x outvocab size 
            y_pred = torch.argmax(Yhat, dim = 2) #batch_size x seq_length x 1
            y_pred = y_pred.view(-1)

            pad_mask   = (y_gold != pad_idx)
            pos_mask   = (y_pred != datagenerator.output_sym2idx['O'])
            neg_mask   = (y_pred == datagenerator.output_sym2idx['O'])

            true_pos  += torch.sum((y_pred==y_gold)*pos_mask*pad_mask)
            true_neg  += torch.sum((y_pred == y_gold)*neg_mask*pad_mask)
            false_pos += torch.sum((y_pred != y_gold)*pos_mask*pad_mask)
            false_neg += torch.sum((y_pred != y_gold)*neg_mask*pad_mask)

        precision = true_pos/(true_pos+false_pos)
        recall    = true_pos/(true_pos+false_neg)
        f_score   = 2*((precision*recall)/(precision+recall))
        return precision, recall, f_score 

        
      def train(self,traingenerator,validgenerator,epochs,batch_size,device='cpu'): 

        self.minloss = 10000000 # the min loss found so far on validation data
        optimizer = torch.optim.Adam(self.parameters())
        
        device    = torch.device(device)
        pad_index = traingenerator.input_sym2idx[traingenerator.pad_token]
        loss_fnc  = nn.CrossEntropyLoss(ignore_index=pad_index)

        for epoch in range(epochs):

          self.training = True  # Tells PyTorch we are in training mode (so dropout is activated).

          epoch_loss = []
          batch_accuracies = []
          for seqX,seqY in traingenerator.generate_batches(batch_size):

            optimizer.zero_grad()
            X = torch.LongTensor(seqX).to(device)
            # Y is of shape batch_size x seq_length
            Y = torch.LongTensor(seqY).to(device).view(-1)

            # Yhat is of shape batch_size x seq_length x nb_classes
            Yhat = self.forward(X)
            num_inputs, seq_length, _ = Yhat.shape
            Yhat = Yhat.view(num_inputs*seq_length, -1)

            loss = loss_fnc(Yhat, Y)
            loss.backward()
            epoch_loss.append(loss.item())

            optimizer.step() # Updates the parameters

            #Accuracy computation
            mask    = (Y != pad_index)
            Yargmax = torch.argmax(Yhat,dim=1)
            correct = torch.sum((Yargmax == Y) * mask)
            total   = torch.sum(mask)
            batch_accuracies.append(float(correct)/float(total))

          if self.verbose:
            print(f"\nEND OF EPOCH {epoch}")
            print(f"[train] mean Loss = {sum(epoch_loss) / len(epoch_loss):.6f} | mean accurracy = {sum(batch_accuracies) / len(batch_accuracies):.6f}")

          # Evaluation
          # This will automatically save the model with minimum loss
          valid_loss, valid_accuracy = self.validate(validgenerator,
                                                     batch_size, 
                                                     device=device,
                                                     save_min_model=True)
          
          # Early stopping
          if self.early_stopping(valid_loss, valid_accuracy):
            return self.early_stopping.best_accuracy, epoch

        # The train function returns the best dev accuracy 
        # and the number of epochs it trained for.
        return self.early_stopping.best_accuracy, epoch


      def validate(self,datagenerator,batch_size,device='cpu',save_min_model=False):

          self.training = False # Tells PyTorch we are in evaluation/inference mode (so dropout is deactivated).
          
          batch_accuracies = []
          wrongs = []
          gold_is = []
          batch_losses      = []

          device = torch.device(device)
          pad_index = datagenerator.input_sym2idx[datagenerator.pad_token]
          loss_fnc  = nn.CrossEntropyLoss(ignore_index=pad_index)

          for (seqX,seqY) in datagenerator.generate_batches(batch_size):
                with torch.no_grad():   
                  X = torch.LongTensor(seqX).to(device)
                  Y = torch.LongTensor(seqY).to(device).view(-1)
                
                  Yhat = self.forward(X)
                  num_inputs, seq_length, _ = Yhat.shape
                  Yhat = Yhat.view(num_inputs*seq_length, -1)

                  loss = loss_fnc(Yhat,Y)
                  batch_losses.append(loss.item())

                  #Accurracy computation
                  mask    = (Y != pad_index)
                  Yargmax = torch.argmax(Yhat,dim=1)
                  correct = torch.sum((Yargmax == Y) * mask)
                  wrong = torch.sum((Yargmax != Y) * mask)
                  gold_i = torch.sum(Y==datagenerator.output_sym2idx['I'])
                  gold_is.append(gold_i)
                  total   = torch.sum(mask)
                  batch_accuracies.append(float(correct)/float(total))
                  wrongs.append(wrong)

          L = len(batch_losses)                  
          valid_loss = sum(batch_losses)/L
          valid_accuracy = sum(batch_accuracies)/L

          if save_min_model and valid_loss < self.minloss:
            self.minloss = valid_loss
            torch.save(self.state_dict(), 'tagger_params.pt')

          if self.verbose:
            print('[valid] mean Loss = %f | mean accurracy = %f'%(valid_loss,valid_accuracy))
            #print(f'wrongs : {sum(wrongs)}\n I tags: {sum(gold_is)}')

          return valid_loss, valid_accuracy # used for early stopping in train

In [ ]:
# PLAIN NER

embed_size = 64
hidden_size = 128
epochs = 50
batch_size = 64

trainset = DataGenerator(train_NER,parentgenerator=grandparent_loader)
validset = DataGenerator(dev_NER,parentgenerator=grandparent_loader)
tagger   = NERtagger(trainset,embed_size,hidden_size,bidirectional=True,device='cuda',patience=5)
tagger.train(trainset,validset,epochs,batch_size,device='cuda') 

p, r, f = tagger.evaluate(validset, device='cuda')
print(f'precision: {p}, recall: {r}, f1: {f}')


END OF EPOCH 0
[train] mean Loss = 0.447250 | mean accurracy = 0.925080
[valid] mean Loss = 0.600456 | mean accurracy = 0.876249

END OF EPOCH 1
[train] mean Loss = 0.230421 | mean accurracy = 0.937995
[valid] mean Loss = 0.562577 | mean accurracy = 0.877209

END OF EPOCH 2
[train] mean Loss = 0.186143 | mean accurracy = 0.937901
[valid] mean Loss = 0.542741 | mean accurracy = 0.876335

END OF EPOCH 3
[train] mean Loss = 0.154037 | mean accurracy = 0.940049
[valid] mean Loss = 0.468003 | mean accurracy = 0.883263

END OF EPOCH 4
[train] mean Loss = 0.119064 | mean accurracy = 0.956662
[valid] mean Loss = 0.380719 | mean accurracy = 0.915817

END OF EPOCH 5
[train] mean Loss = 0.090411 | mean accurracy = 0.971784
[valid] mean Loss = 0.314255 | mean accurracy = 0.933582

END OF EPOCH 6
[train] mean Loss = 0.069383 | mean accurracy = 0.977729
[valid] mean Loss = 0.262234 | mean accurracy = 0.938919

END OF EPOCH 7
[train] mean Loss = 0.054381 | mean accurracy = 0.982958
[valid] mean Loss

In [ ]:
# FINE TUNING

epochs = 50
batch_size = 64

trainset = DataGenerator(train_speaker,parentgenerator=grandparent_loader)
validset = DataGenerator(dev_speaker,parentgenerator=grandparent_loader)
testset  = DataGenerator(test_speaker,parentgenerator=grandparent_loader)
tagger.train(trainset,validset,epochs,batch_size,device='cuda') 
tagger.validate(testset, batch_size = 64, device = 'cuda')

p, r, f = tagger.evaluate(testset, device='cuda')
print(f'precision: {p}, recall: {r}, f1: {f}')


END OF EPOCH 0
[train] mean Loss = 0.152316 | mean accurracy = 0.965370
[valid] mean Loss = 0.061831 | mean accurracy = 0.989794

END OF EPOCH 1
[train] mean Loss = 0.088675 | mean accurracy = 0.980783
[valid] mean Loss = 0.047394 | mean accurracy = 0.990142

END OF EPOCH 2
[train] mean Loss = 0.063773 | mean accurracy = 0.980198
[valid] mean Loss = 0.039300 | mean accurracy = 0.987690

END OF EPOCH 3
[train] mean Loss = 0.055155 | mean accurracy = 0.981433
[valid] mean Loss = 0.034806 | mean accurracy = 0.990352

END OF EPOCH 4
[train] mean Loss = 0.047722 | mean accurracy = 0.984995
[valid] mean Loss = 0.032241 | mean accurracy = 0.991244

END OF EPOCH 5
[train] mean Loss = 0.042425 | mean accurracy = 0.987078
[valid] mean Loss = 0.029950 | mean accurracy = 0.990965

END OF EPOCH 6
[train] mean Loss = 0.039014 | mean accurracy = 0.987534
[valid] mean Loss = 0.028292 | mean accurracy = 0.991229

END OF EPOCH 7
[train] mean Loss = 0.036277 | mean accurracy = 0.988584
[valid] mean Loss

# Statistics on the data

In [ ]:
# Most frequent class baseline (accuracy)
test_speaker['tags'].apply(lambda x: x.count('O')/len(x)).mean()

0.9879597287772806

In [ ]:
# Precision baseline
test_speaker['tags'].apply(lambda x: (x.count('I') + x.count('B'))/len(x)).mean()

0.012040271222719277

In [ ]:

print('total number of examples: ', df_speaker.shape[0])

print('number of examples where speaker is identified: ', df_speaker[df_speaker.speaker != "None"].shape[0])

print('number of examples where speaker is identified in text: ', df_speaker['tags'].apply(lambda x: x.count('I') != 0).sum())

total number of examples:  688
number of examples where speaker is identified:  279
number of examples where speaker is identified in text:  240


In [ ]:
#example of sentence when the speaker is identified but nor present 
#our system is not able to predict speaker for these

df_speaker['not_none'] = df_speaker['speaker'].apply(lambda x: x != "None")

df_speaker['only_O'] = df_speaker['tags'].apply(lambda x: x.count('I') + x.count('B') == 0)

for ind in df_speaker[df_speaker.not_none][df_speaker.only_O].index:
  print('text: ', df_speaker['text'][ind])
  print('\nspeaker:\n', df_speaker['speaker'][ind])
  print('---------------------------------------------------------')
  break

text:  Jeudi, sur France 2, la candidate RN déclarait que " la peine de mort pourrait passer par un référendum " puisque " tout pourrait passer par un référendum ". Ce vendredi, sur BFM, elle est revenue sur ses propos : " C'est anticonstitutionnel ".

speaker:
 Marine Le Pen
---------------------------------------------------------


<ipython-input-27-e9df4b320948>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for ind in df_speaker[df_speaker.not_none][df_speaker.only_O].index:


In [ ]:
df_speaker['tokenized_text'].apply(lambda x: len(x)).mean()

89.29796511627907